# 官网教程- Quickstart 笔记
[官网](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)
## 数据
PyTorch 提供两个数据相关的类：**torch.utils.data.DataLoader**、**torch.utils.data.Dataset**  
**torch.utils.data.Dataset**: 官方提供的一些开源数据。  
**torch.utils.data.DataLoader**: 数据处理迭代器，可提供常用是数据处理操作。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## 领域库
PyTorch 提供不同领域的库：[TorchText](https://pytorch.org/text/stable/index.html)、[TorchVision](https://pytorch.org/vision/stable/index.html)、[TorchAudio](https://pytorch.org/audio/stable/index.html)

## 下载数据

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



# 数据处理
将数据传给**DataLoader**进行数据处理

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 创建模型
1. 创建模型类，该类继承与**nn.Module**;
2. 在**__init__**中定义需要的网络层；
3. 在**forware**中实现网络结构(前向传播）；
4. 如果GPU可用的话将模型载入GPU中。

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 损失函数和优化器

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## 训练
### 一次数据训练的训练流程(loop)
1. 模型预测；
2. 反向传播

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## 通过测试数据检测模型性能

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 进行多轮数据训练

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314386  [    0/60000]
loss: 2.301813  [ 6400/60000]
loss: 2.283430  [12800/60000]
loss: 2.271753  [19200/60000]
loss: 2.249379  [25600/60000]
loss: 2.223309  [32000/60000]
loss: 2.234460  [38400/60000]
loss: 2.199915  [44800/60000]
loss: 2.194132  [51200/60000]
loss: 2.169718  [57600/60000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 2.162025 

Epoch 2
-------------------------------
loss: 2.175573  [    0/60000]
loss: 2.165338  [ 6400/60000]
loss: 2.107556  [12800/60000]
loss: 2.116529  [19200/60000]
loss: 2.065055  [25600/60000]
loss: 2.010356  [32000/60000]
loss: 2.038589  [38400/60000]
loss: 1.960442  [44800/60000]
loss: 1.961038  [51200/60000]
loss: 1.897536  [57600/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.893170 

Epoch 3
-------------------------------
loss: 1.931176  [    0/60000]
loss: 1.899253  [ 6400/60000]
loss: 1.782973  [12800/60000]
loss: 1.815587  [19200/60000]
loss: 1.705374  [25600/60000]
loss: 1.661443  [32000/600